# Objective: Predict % of Mentees that might leave Underdog Devs

### What metrics would be helpful to predict this?

- Number of support (customer service) tickets created
- Length of time with Underdog Devs
- Sessions attended overall
- Sessions attended per week
- Time commitment (suggested intake data point from Nigel's excel sheet)
- Days available (suggested intake data point from Nigel's excel sheet)
- ... Plus more that I haven't thought of...

### Why is this relevant?

- Admin could anticipate when mentees might leave (and how many people might leave per month). This could help so admin is not left scrambling trying to solve mentor/mentee matching issues
- This is probably not super relevant at the moment, but perhaps if Underdog Devs has many hundreds (or thousands) of mentees in the future then this data could be more useful.


In [44]:
# Generate Mock Data

import random as r
import pandas as pd
import datetime


def random_date(start_y, start_m, start_d, end_y, end_m, end_d):
    start_date = datetime.date(start_y, start_m, start_d)
    end_date = datetime.date(end_y, end_m, end_d)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = r.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    return random_date


def percent_true(percent: int) -> bool:
    return r.randint(1, 100) <= percent


def get_status():
    if percent_true(90):
        if percent_true(55):
            return 'graduated'
        else:
            return 'active'
    return 'left'


def get_date_joined(status):
    if status == 'left':
        return random_date(2020, 1, 1, 2021, 11, 1)
    if status == 'graduated':
        return random_date(2020, 1, 1, 2021, 4, 1)
    if status == 'active':
        return random_date(2021, 9, 1, 2022, 1, 29)


def get_cst_val(status):  # purposefully skewing data
    if status == 'graduated':
        return r.randint(1, 4)
    if status == 'active':
        return r.randint(1, 2)
    return r.randint(3, 10)


def hrs(status):  # purposefully skewing data
    if status == 'graduated':
        return r.randrange(15, 40, 5)
    if status == 'active':
        return r.randrange(10, 40, 5)
    return r.randrange(5, 30, 5)


def sessions(status):  # purposefully skewing data
    if status == 'graduated':
        return r.randint(10, 40)
    if status == 'active':
        return r.randint(15, 20)
    return r.randint(1, 15)


def no_days(status):  # purposefully skewing data
    if status == 'graduated':
        return r.randint(2, 6)
    if status == 'active':
        return r.randint(3, 6)
    return r.randint(1, 3)


def departure_date(date_joined, status, no_sessions):
    if status == 'graduated':
        days = r.randint(90, 240)
        duration = datetime.timedelta(days=days)
        departure_date = date_joined + duration
        return departure_date
    if status == 'active':
        return None
    if 0 < no_sessions <= 2:
        days = r.randint(14, 42)
    elif 2 < no_sessions <= 4:
        days = r.randint(28, 54)
    elif 4 < no_sessions <= 6:
        days = r.randint(42, 70)
    elif 6 < no_sessions <= 8:
        days = r.randint(56, 84)
    elif 8 < no_sessions <= 10:
        days = r.randint(70, 112)
    elif 10 < no_sessions <= 15:
        days = r.randint(84, 140)
    duration = datetime.timedelta(days=days)
    departure_date = date_joined + duration
    return departure_date


class Mentee:

    def __init__(self):
        self.profile_id = 'E' + str(r.randint(1000000, 70000000000000))
        self.status = get_status()
        self.date_joined = get_date_joined(self.status)
        self.cust_serv_tickets = get_cst_val(self.status)
        self.time_commitment = hrs(self.status)  # hours/week
        self.no_sessions = sessions(self.status)
        self.days_available = no_days(self.status)  # days/week, max is Mon-Sat
        self.departure_date = departure_date(self.date_joined,
                                             self.status,
                                             self.no_sessions)

    @classmethod
    def to_df(cls, num_rows):
        return pd.DataFrame(vars(cls()) for _ in range(num_rows))


In [45]:
mentee = Mentee()
mentee_df = mentee.to_df(5000)
mentee_df[mentee_df['status'] == 'left'].head()


,profile_id,status,date_joined,cust_serv_tickets,time_commitment,no_sessions,days_available,departure_date
31,E46910332965041,left,2021-10-20,5,10,13,3,2022-02-16
37,E11231814197623,left,2020-07-06,10,10,7,1,2020-09-08
52,E22236467653399,left,2020-02-18,7,5,10,2,2020-05-01
60,E14326287789403,left,2020-09-02,3,20,2,2,2020-10-14
86,E59007439982229,left,2020-02-07,10,25,9,1,2020-05-05


In [46]:
mentee_df['status'].value_counts()


graduated    2417
active       2075
left          508
Name: status, dtype: int64

# Feature Engineer a Duration Column

In [47]:
year = datetime.datetime.today().year
month = datetime.datetime.today().month
day = datetime.datetime.today().day
today = datetime.date(year=year, month=month, day=day)
mentee_df['today'] = today
mentee_df.head()


,profile_id,status,date_joined,cust_serv_tickets,time_commitment,no_sessions,days_available,departure_date,today
0,E12819469359859,graduated,2020-05-30,1,30,32,3,2020-12-10,2022-02-03
1,E50425127055070,graduated,2020-04-22,2,35,22,2,2020-12-09,2022-02-03
2,E50942948501269,graduated,2020-04-16,2,30,10,4,2020-08-25,2022-02-03
3,E46787968659140,graduated,2021-01-19,3,15,14,3,2021-06-15,2022-02-03
4,E53232344814528,graduated,2020-03-28,4,20,31,5,2020-07-22,2022-02-03


In [48]:
def get_duration(row):
    if row['status'] == 'active':
        duration = row['today'] - row['date_joined']
    else:
        duration = row['departure_date'] - row['date_joined']
    return duration

mentee_df['duration'] = mentee_df.apply(get_duration, axis=1)
mentee_df.head()


,profile_id,status,date_joined,cust_serv_tickets,time_commitment,no_sessions,days_available,departure_date,today,duration
0,E12819469359859,graduated,2020-05-30,1,30,32,3,2020-12-10,2022-02-03,194 days
1,E50425127055070,graduated,2020-04-22,2,35,22,2,2020-12-09,2022-02-03,231 days
2,E50942948501269,graduated,2020-04-16,2,30,10,4,2020-08-25,2022-02-03,131 days
3,E46787968659140,graduated,2021-01-19,3,15,14,3,2021-06-15,2022-02-03,147 days
4,E53232344814528,graduated,2020-03-28,4,20,31,5,2020-07-22,2022-02-03,116 days


# We will need 2 separate models to compute attrition rate

- First, a classification model for the active mentees
- Second, a regression model that predicts duration

# Model 1: Classification

In [49]:
def make_y_val(row):
    if row['status'] == 'active':
        num = 2
    if row['status'] == 'graduated':
        num = 1
    if row['status'] == 'left':
        num = 0
    return num


In [50]:
mentee_df_all_c = mentee_df.copy()  # '_c' for classification
mentee_df_all_c = mentee_df_all_c[['cust_serv_tickets', 'no_sessions',
                                   'time_commitment', 'days_available',
                                   'status']]

# Create numerical y value
mentee_df_all_c['y'] = mentee_df_all_c.apply(make_y_val, axis=1)
mentee_df_all_c.drop(columns='status', inplace=True)

# Drop rows of 'active' mentees
mentee_df_c = mentee_df_all_c[mentee_df_all_c['y'] != 2]
mentee_df_c['y'].value_counts()


1    2417
0     508
Name: y, dtype: int64

In [51]:
mentee_df_c.dtypes


cust_serv_tickets    int64
no_sessions          int64
time_commitment      int64
days_available       int64
y                    int64
dtype: object

In [52]:
mentee_df_c.head(3)


,cust_serv_tickets,no_sessions,time_commitment,days_available,y
0,1,32,30,3,1
1,2,22,35,2,1
2,2,10,30,4,1


## Check Baseline Accuracy

- i.e. always predict the mentee will graduate

In [53]:
print('Baseline Accuracy:',
      mentee_df_c['y'].value_counts(normalize=True).max())


Baseline Accuracy: 0.8263247863247863


In [54]:
# Create X and y
y = mentee_df_c['y']
X = mentee_df_c.drop(columns='y')


## Split into Train and Validation Set

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)


In [56]:
y_train.value_counts()


1    1930
0     410
Name: y, dtype: int64

In [57]:
y_val.value_counts()


1    487
0     98
Name: y, dtype: int64

## Build Model

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
rfc = RandomForestClassifier()

param_grid = {
    "max_depth": [15, 20, 25, 30, 35, 40, 45, 50],
    "n_estimators": [10, 20, 30]
}

rfc_model = RandomizedSearchCV(rfc,
                               param_distributions=param_grid,
                               n_iter=15,
                               n_jobs=-1,
                               cv=3,
                               verbose=1)


In [59]:
rfc_model.fit(X_train, y_train)


Fitting 3 folds for each of 15 candidates, totalling 45 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=15,
                   n_jobs=-1,
                   param_distributions={'max_depth': [15, 20, 25, 30, 35, 40,
                                                      45, 50],
                                        'n_estimators': [10, 20, 30]},
                   verbose=1)

In [60]:
rfc_model.best_score_


0.9901709401709402

In [61]:
rfc_model.best_params_


{'max_depth': 15, 'n_estimators': 20}

## Test Model On Held Back Data (validation set)

In [62]:
from sklearn.metrics import accuracy_score
val_predictions = rfc_model.predict(X_val)
accuracy_score(val_predictions, y_val)


0.9914529914529915

## Filter data to 'active' mentees

In [63]:
# 2: active, 1: graduated, 0: left
mentee_df_all_c['y'].value_counts()


1    2417
2    2075
0     508
Name: y, dtype: int64

In [64]:
mentee_df_active = mentee_df_all_c[mentee_df_all_c['y'] == 2].copy()
mentee_df_active['y'].value_counts()


2    2075
Name: y, dtype: int64

In [65]:
mentee_df_active.head()


,cust_serv_tickets,no_sessions,time_commitment,days_available,y
8,1,20,20,4,2
9,1,18,20,6,2
10,1,18,10,3,2
11,2,15,30,4,2
14,2,16,20,4,2


## Use classification model to predict the outcome of the active mentees

In [66]:
X = mentee_df_active.drop(columns='y')
pred_outcome = rfc_model.predict(X)
mentee_df_active['pred_outcome'] = pred_outcome
mentee_df_active.head()


,cust_serv_tickets,no_sessions,time_commitment,days_available,y,pred_outcome
8,1,20,20,4,2,1
9,1,18,20,6,2,1
10,1,18,10,3,2,0
11,2,15,30,4,2,1
14,2,16,20,4,2,1


In [67]:
mentee_df_active['pred_outcome'].value_counts()


1    1712
0     363
Name: pred_outcome, dtype: int64

# Model 2: Regression

## First, build a model based on the mentees who left, to predict how long they stay before leaving

In [68]:
mentee_df_left = mentee_df[mentee_df['status'] == 'left'].copy()
mentee_df_left.head()


,profile_id,status,date_joined,cust_serv_tickets,time_commitment,no_sessions,days_available,departure_date,today,duration
31,E46910332965041,left,2021-10-20,5,10,13,3,2022-02-16,2022-02-03,119 days
37,E11231814197623,left,2020-07-06,10,10,7,1,2020-09-08,2022-02-03,64 days
52,E22236467653399,left,2020-02-18,7,5,10,2,2020-05-01,2022-02-03,73 days
60,E14326287789403,left,2020-09-02,3,20,2,2,2020-10-14,2022-02-03,42 days
86,E59007439982229,left,2020-02-07,10,25,9,1,2020-05-05,2022-02-03,88 days


In [69]:
# Convert duration to days, this is our y value
mentee_df_left['duration'] = mentee_df_left['duration'].dt.days
mentee_df_left.head()


,profile_id,status,date_joined,cust_serv_tickets,time_commitment,no_sessions,days_available,departure_date,today,duration
31,E46910332965041,left,2021-10-20,5,10,13,3,2022-02-16,2022-02-03,119
37,E11231814197623,left,2020-07-06,10,10,7,1,2020-09-08,2022-02-03,64
52,E22236467653399,left,2020-02-18,7,5,10,2,2020-05-01,2022-02-03,73
60,E14326287789403,left,2020-09-02,3,20,2,2,2020-10-14,2022-02-03,42
86,E59007439982229,left,2020-02-07,10,25,9,1,2020-05-05,2022-02-03,88


In [70]:
mentee_df_left = mentee_df_left[['cust_serv_tickets', 'time_commitment',
                                 'no_sessions', 'days_available', 'duration']]

# Create X and y
y = mentee_df_left['duration']
X = mentee_df_left.drop(columns='duration')


## Build model to predict mentee duration based on mentees who left

In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)


In [72]:
X_train.head()


,cust_serv_tickets,no_sessions,time_commitment,days_available
2894,6,12,10,1
3996,2,19,25,6
810,1,37,30,5
1293,3,39,25,4
1561,8,10,25,2


In [73]:
poly_features = poly.fit_transform(X)


In [74]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)


In [75]:
poly_reg_model = LinearRegression()
poly_reg_model.fit(X_train, y_train)


LinearRegression()

## Test Model On Held Back Data (validation set)

In [76]:
from sklearn import metrics
import numpy as np
val_predictions = poly_reg_model.predict(X_val)
print('Mean Absolute Error (MAE):',
      metrics.mean_absolute_error(val_predictions, y_val))
print('R Squared:', metrics.r2_score(val_predictions, y_val))


Mean Absolute Error (MAE): 11.12258171146837
R Squared: 0.7728054817446152


### Basically, the model's predictions are off by about __ [MAE] __ on average.
- MAE will change with each runtime but in my case it says 9.8

In [77]:
poly_reg_model.coef_


array([-0.08610166, -0.03228614,  6.85299406,  0.49297142])

## Now we filter the earlier dataframe for mentees who are predicted to leave

In [78]:
# Currently active mentees that the classification
# model predicts will leave
mentee_df_leaving_predictions = \
    mentee_df_active[mentee_df_active['pred_outcome'] == 0].copy()

mentee_df_leaving_predictions.head()


,cust_serv_tickets,no_sessions,time_commitment,days_available,y,pred_outcome
10,1,18,10,3,2,0
57,1,18,10,4,2,0
73,1,20,10,4,2,0
93,2,15,10,4,2,0
103,2,17,10,5,2,0


## Now apply the regression model to predict duration

In [79]:
mentee_df_leaving_predictions = \
    mentee_df_leaving_predictions[['cust_serv_tickets',
                                   'time_commitment',
                                   'no_sessions',
                                   'days_available']].copy()

duration_pred = poly_reg_model.predict(mentee_df_leaving_predictions)
mentee_df_leaving_predictions['duration_pred'] = duration_pred
mentee_df_leaving_predictions.head()


,cust_serv_tickets,time_commitment,no_sessions,days_available,duration_pred
10,1,10,18,3,144.223344
57,1,10,18,4,144.716316
73,1,10,20,4,158.422304
93,2,10,15,4,124.071232
103,2,10,17,5,138.270191


## Now we add the predicted duration onto each mentee's start date

In [80]:
mentee_df.head()


,profile_id,status,date_joined,cust_serv_tickets,time_commitment,no_sessions,days_available,departure_date,today,duration
0,E12819469359859,graduated,2020-05-30,1,30,32,3,2020-12-10,2022-02-03,194 days
1,E50425127055070,graduated,2020-04-22,2,35,22,2,2020-12-09,2022-02-03,231 days
2,E50942948501269,graduated,2020-04-16,2,30,10,4,2020-08-25,2022-02-03,131 days
3,E46787968659140,graduated,2021-01-19,3,15,14,3,2021-06-15,2022-02-03,147 days
4,E53232344814528,graduated,2020-03-28,4,20,31,5,2020-07-22,2022-02-03,116 days


In [81]:
leaving_predictions = list(mentee_df_leaving_predictions.index.values)
mentee_df['duration_pred'] = ''
for i in range(len(mentee_df)):
    if i in leaving_predictions:
        duration_pred_ = mentee_df_leaving_predictions.at[i, 'duration_pred']
        date_joined = mentee_df.at[i, 'date_joined']
        mentee_df_leaving_predictions.at[i, 'pred_leave_date'] = \
            date_joined + datetime.timedelta(days=duration_pred_)


In [82]:
mentee_df_leaving_predictions.head()


,cust_serv_tickets,time_commitment,no_sessions,days_available,duration_pred,pred_leave_date
10,1,10,18,3,144.223344,2022-03-30
57,1,10,18,4,144.716316,2022-02-28
73,1,10,20,4,158.422304,2022-03-03
93,2,10,15,4,124.071232,2022-01-12
103,2,10,17,5,138.270191,2022-03-09


## How many mentees are predicted to leave in March 2022?

In [83]:
mentee_df_leaving_predictions['pred_leave_date'] = \
    pd.to_datetime(mentee_df_leaving_predictions['pred_leave_date'])

no_mentees_pred_to_leave_each_month = \
    mentee_df_leaving_predictions.groupby(pd.Grouper(key='pred_leave_date',
                                                     freq='M')).count()

no_mentees_pred_to_leave_each_month


,cust_serv_tickets,time_commitment,no_sessions,days_available,duration_pred
pred_leave_date,,,,,
2022-01-31,17,17,17,17,17
2022-02-28,59,59,59,59,59
2022-03-31,74,74,74,74,74
2022-04-30,75,75,75,75,75
2022-05-31,75,75,75,75,75
2022-06-30,61,61,61,61,61
2022-07-31,2,2,2,2,2


- Each row is 1 month
- The count, which is shown in each column, is the total number of mentees that are predicted to leave in that particular month
- Data will change with each runtime, but it currently shows 72 mentees will leave in March 2022

In [84]:
# Total active mentees
total_active_mentees = mentee_df_active.shape[0]
total_active_mentees


2075

In [85]:
march_number = no_mentees_pred_to_leave_each_month.iloc[2, 1]
march_number


74

# Attrition for March 2022

In [86]:
attrition = march_number / total_active_mentees
attrition = "{:.2%}".format(attrition)
attrition


'3.57%'

### Overall, the mock data and models are not perfect. The main point of the notebook is more to provide a rough outline for how the process could look in the future.